##colab上での操作時に使用

In [ ]:
!pip install lightgbm

ERROR: Invalid requirement: 'lightgbm:latest'


In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd /content/drive/MyDrive/MyApps/HorseRacingApp/app_azure_realtime/notebook
except:
    pass

Mounted at /content/drive
/content/drive/MyDrive/MyApps/HorseRacingApp/app_azure_realtime/notebook


## import requred libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pickle # 完成したデータを保存する用
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import TimeSeriesSplit, GroupShuffleSplit
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
import json
import matplotlib.pyplot as plt
import seaborn as sns

## Database

In [5]:
from sqlalchemy import create_engine
%cd /content/drive/MyDrive/MyApps/HorseRacingApp/app_azure_realtime
engine = create_engine('sqlite:///data.db')

/content/drive/MyDrive/MyApps/HorseRacingApp/app_azure_realtime


In [6]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base(bind=engine)

In [7]:
from sqlalchemy.orm import sessionmaker
SessionClass = sessionmaker(engine)  # セッションを作るクラスを作成
session = SessionClass()

In [9]:
class MainData(Base):
    __tablename__ = 'maindata'
    __table_args__ = {'autoload':True}

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.MainData, and will be replaced in the string-lookup table.
  """Entry point for launching an IPython kernel.


In [10]:
data = session.query(MainData).all()

In [11]:
#Custom function to convert the data
def serialize(data): #queryしたデータをjson形式にするため
    """Return object data in easily serializable format"""
    return {
        'Race_id': data.Race_id,
        'Race_date':data.Race_date,
        'Race_Name': data.Race_Name,
        'Ranking': data.Ranking,
        'Uni_Num': data.Uni_Num,
        'Hor_Num': data.Hor_Num,
        'Hor_name': data.Hor_name,
        'Hor_sex_and_age': data.Hor_sex_and_age,
        'JockeyWeight': data.JockeyWeight,
        'Jockey': data.Jockey,
        'Race_Time': data.Race_Time,
        'Popularity': data.Popularity,
        'Odds_popularity': data.Odds_popularity,
        'ato_3_F': data.ato_3_F,
        'Trainer_1': data.Trainer_1,
        'Trainer_2': data.Trainer_2,
        'Horse_weight': data.Horse_weight,
        'Horse_weight_Flux': data.Horse_weight_Flux,
        'start_time': data.start_time,
        'weather': data.weather,
        'field_condition': data.field_condition,
        'competition_count': data.competition_count,
        'venue_area': data.venue_area,
        'day_count': data.day_count,
        'horse_class': data.horse_class,
        'horse_class_2': data.horse_class_2,
        'race_type': data.race_type,
        'hadicap': data.hadicap,
        'number_of_hourses': data.number_of_hourses,
        'field_length': data.field_length,
        'field_type_1': data.field_type_1,
        'field_type_2': data.field_type_2,
        'corner_rank_1': data.corner_rank_1,
        'corner_rank_2': data.corner_rank_2,
        'corner_rank_3': data.corner_rank_3,
        'corner_rank_4': data.corner_rank_4,
        'target': data.target,
    }

In [15]:
records = []
for i in data:
    record = i.__dict__
    records.append(record)

In [16]:
df = pd.DataFrame(records)

In [18]:
df = df.drop('_sa_instance_state',axis=1)
df

,Uni_num,field_condition,ato_3_F,Corner_rank_3,Hor_Num,competition_count,Trainer,Corner_rank_4,Hor_name,venue_area,Race_id,Horse_weight_Flux,Horse_weight,Hor_sex_and_age,day_count,Race_date,single_odds,field_length,Corner_rank_2,JockeyWeight,horse_class,Race_name,start_time,field_type_1,Jockey,number_of_horses,Ranking,weather,field_type_2,record_id,Race_Time,Corner_rank_1,Odds_popularity
0,4,良,38.9,NaN,7,1,美浦金成,NaN,オイデヤスダイジン,中山,202006010101,2.0,460.0,牝3,1,20200105,3.6,1200.0,5.0,54.0,サラ系３歳,3歳未勝利,955.0,ダ,伊藤,16,1,晴,右,1,73.3,5,2.0
1,1,良,38.0,NaN,1,1,美浦林,NaN,グルアーブ,中山,202006010101,4.0,432.0,牝3,1,20200105,17.2,1200.0,11.0,54.0,サラ系３歳,3歳未勝利,955.0,ダ,柴田善,16,2,晴,右,2,73.3,12,5.0
2,2,良,38.7,NaN,3,1,美浦萱野,NaN,トーレスデルパイネ,中山,202006010101,0.0,458.0,牝3,1,20200105,199.0,1200.0,9.0,54.0,サラ系３歳,3歳未勝利,955.0,ダ,岩部,16,3,晴,右,3,73.7,9,12.0
3,3,良,39.2,NaN,6,1,美浦星野,NaN,デュープス,中山,202006010101,0.0,418.0,牝3,1,20200105,50.4,1200.0,6.0,54.0,サラ系３歳,3歳未勝利,955.0,ダ,武士沢,16,4,晴,右,4,73.8,6,8.0
4,6,良,39.7,NaN,12,1,美浦加藤征,NaN,ディヴィニティ,中山,202006010101,-2.0,536.0,牝3,1,20200105,1.6,1200.0,4.0,54.0,サラ系３歳,3歳未勝利,955.0,ダ,マーフ,16,5,晴,右,5,73.8,4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92905,2,重,36.9,NaN,3,6,美浦久保田,NaN,ヤマメ,中京,202107060512,-2.0,474.0,牝4,5,20211218,226.6,1400.0,11.0,55.0,サラ系３歳以上,大須特別,1610.0,ダ,藤井勘,16,11,晴,左,92906,84.6,11,14.0
92906,3,重,38.7,NaN,5,6,栗東武英,NaN,ロンドンデリーエア,中京,202107060512,6.0,470.0,牡4,5,20211218,21.4,1400.0,2.0,57.0,サラ系３歳以上,大須特別,1610.0,ダ,富田,16,12,晴,左,92907,84.8,2,7.0
92907,2,重,38.0,NaN,4,6,栗東今野,NaN,ナリス,中京,202107060512,-2.0,476.0,牝5,5,20211218,86.9,1400.0,8.0,55.0,サラ系３歳以上,大須特別,1610.0,ダ,荻野琢,16,13,晴,左,92908,84.9,8,11.0
92908,7,重,38.1,NaN,13,6,美浦中野,NaN,サノハニー,中京,202107060512,14.0,488.0,牝4,5,20211218,213.0,1400.0,13.0,55.0,サラ系３歳以上,大須特別,1610.0,ダ,小林凌,16,14,晴,左,92909,86.3,13,13.0


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92910 entries, 0 to 92909
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Uni_num            92910 non-null  int64  
 1   field_condition    92910 non-null  object 
 2   ato_3_F            92910 non-null  float64
 3   Corner_rank_3      45162 non-null  float64
 4   Hor_Num            92910 non-null  int64  
 5   competition_count  92910 non-null  int64  
 6   Trainer            92910 non-null  object 
 7   Corner_rank_4      39573 non-null  float64
 8   Hor_name           92910 non-null  object 
 9   venue_area         92910 non-null  object 
 10  Race_id            92910 non-null  int64  
 11  Horse_weight_Flux  92893 non-null  float64
 12  Horse_weight       92910 non-null  float64
 13  Hor_sex_and_age    92910 non-null  object 
 14  day_count          92910 non-null  int64  
 15  Race_date          92910 non-null  int64  
 16  single_odds        929

In [21]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Uni_num,92910.0,4.762458e+00,2.270424e+00,1.000000e+00,3.000000e+00,5.000000e+00,7.000000e+00,8.000000e+00
ato_3_F,92910.0,3.652143e+01,4.630113e+00,1.280000e+01,3.540000e+01,3.700000e+01,3.860000e+01,1.143000e+02
Corner_rank_3,45162.0,6.867477e+00,4.143978e+00,1.000000e+00,3.000000e+00,6.000000e+00,1.000000e+01,1.800000e+01
Hor_Num,92910.0,7.721612e+00,4.404184e+00,1.000000e+00,4.000000e+00,7.000000e+00,1.100000e+01,1.800000e+01
competition_count,92910.0,2.652599e+00,1.458175e+00,1.000000e+00,1.000000e+00,2.000000e+00,4.000000e+00,6.000000e+00
Corner_rank_4,39573.0,6.853461e+00,4.133992e+00,1.000000e+00,3.000000e+00,6.000000e+00,1.000000e+01,1.800000e+01
Race_id,92910.0,2.020552e+11,5.002219e+07,2.020010e+11,2.020060e+11,2.020100e+11,2.021060e+11,2.021100e+11
Horse_weight_Flux,92893.0,4.165330e-01,6.331920e+00,-4.200000e+01,-4.000000e+00,0.000000e+00,4.000000e+00,6.600000e+01
Horse_weight,92910.0,4.709189e+02,3.018788e+01,3.360000e+02,4.500000e+02,4.720000e+02,4.920000e+02,5.920000e+02
day_count,92910.0,4.705877e+00,2.637183e+00,1.000000e+00,3.000000e+00,4.000000e+00,7.000000e+00,1.200000e+01


## Add more features to the dataframe by computing some statistics

In [22]:
#Label encoding
le = LabelEncoder()
df['Race_id'] = le.fit_transform(df['Race_id'])
df.head()

,Uni_num,field_condition,ato_3_F,Corner_rank_3,Hor_Num,competition_count,Trainer,Corner_rank_4,Hor_name,venue_area,Race_id,Horse_weight_Flux,Horse_weight,Hor_sex_and_age,day_count,Race_date,single_odds,field_length,Corner_rank_2,JockeyWeight,horse_class,Race_name,start_time,field_type_1,Jockey,number_of_horses,Ranking,weather,field_type_2,record_id,Race_Time,Corner_rank_1,Odds_popularity
0,4,良,38.9,NaN,7,1,美浦金成,NaN,オイデヤスダイジン,中山,1404,2.0,460.0,牝3,1,20200105,3.6,1200.0,5.0,54.0,サラ系３歳,3歳未勝利,955.0,ダ,伊藤,16,1,晴,右,1,73.3,5,2.0
1,1,良,38.0,NaN,1,1,美浦林,NaN,グルアーブ,中山,1404,4.0,432.0,牝3,1,20200105,17.2,1200.0,11.0,54.0,サラ系３歳,3歳未勝利,955.0,ダ,柴田善,16,2,晴,右,2,73.3,12,5.0
2,2,良,38.7,NaN,3,1,美浦萱野,NaN,トーレスデルパイネ,中山,1404,0.0,458.0,牝3,1,20200105,199.0,1200.0,9.0,54.0,サラ系３歳,3歳未勝利,955.0,ダ,岩部,16,3,晴,右,3,73.7,9,12.0
3,3,良,39.2,NaN,6,1,美浦星野,NaN,デュープス,中山,1404,0.0,418.0,牝3,1,20200105,50.4,1200.0,6.0,54.0,サラ系３歳,3歳未勝利,955.0,ダ,武士沢,16,4,晴,右,4,73.8,6,8.0
4,6,良,39.7,NaN,12,1,美浦加藤征,NaN,ディヴィニティ,中山,1404,-2.0,536.0,牝3,1,20200105,1.6,1200.0,4.0,54.0,サラ系３歳,3歳未勝利,955.0,ダ,マーフ,16,5,晴,右,5,73.8,4,1.0


In [24]:
"""馬別に統計量を取得"""
horse_df_1 = df[['Hor_name','Odds_popularity','Horse_weight','Corner_rank_1','Corner_rank_2', 'Corner_rank_3', 'Corner_rank_4','field_length','Horse_weight_Flux','JockeyWeight']].groupby('Hor_name').mean()
horse_df_1 = horse_df_1.sort_values('Hor_name')
horse_df_1.columns = [i+('_mean') for i in horse_df_1.columns]
horse_df_1 = horse_df_1.reset_index()
horse_df_2 = df[['Hor_name','Race_id']].groupby('Hor_name').count()
horse_df_2.columns = ['Race_experience_count']
horse_df_2 = horse_df_2.reset_index()

"""## 騎手別に統計量を取得"""
## As it is

"""## フィールドデータなどの取得"""
## As it is

"""##各dfを結合して特徴量を作成する"""
df = df.merge(horse_df_1,how='left',on='Hor_name')
df = df.merge(horse_df_2,how='left',on='Hor_name')
df = df.sort_values('Race_id')

"""カテゴリ変数をlabel encodingで変換する"""
cat_cols = ['Race_name', 'Hor_name','Race_date', 'Hor_sex_and_age', 'Jockey', 'Trainer', 'weather', 'field_condition', 'venue_area', 'horse_class', 'field_type_1', 'field_type_2']
for col in cat_cols:
    le = LabelEncoder()
    le.fit(df[col])
    df[col] = le.transform(df[col])

#最終的な特徴量
df.head()

,Uni_num,field_condition,ato_3_F,Corner_rank_3,Hor_Num,competition_count,Trainer,Corner_rank_4,Hor_name,venue_area,Race_id,Horse_weight_Flux,Horse_weight,Hor_sex_and_age,day_count,Race_date,single_odds,field_length,Corner_rank_2,JockeyWeight,horse_class,Race_name,start_time,field_type_1,Jockey,number_of_horses,Ranking,weather,field_type_2,record_id,Race_Time,Corner_rank_1,Odds_popularity,Odds_popularity_mean_x,Horse_weight_mean_x,Corner_rank_1_mean_x,Corner_rank_2_mean_x,Corner_rank_3_mean_x,Corner_rank_4_mean_x,field_length_mean_x,Horse_weight_Flux_mean_x,JockeyWeight_mean_x,Race_experience_count_x,Odds_popularity_mean_y,Horse_weight_mean_y,Corner_rank_1_mean_y,Corner_rank_2_mean_y,Corner_rank_3_mean_y,Corner_rank_4_mean_y,field_length_mean_y,Horse_weight_Flux_mean_y,JockeyWeight_mean_y,Race_experience_count_y
0,6,2,35.6,3.0,6,1,241,3.0,1418,6,0,4.0,438.0,10,1,59,16.0,1800.0,5.0,54.0,0,2,955.0,1,149,6,1,3,0,27333,109.7,5,3.0,7.727273,450.000000,6.454545,6.545455,6.111111,6.875000,1781.818182,3.636364,53.363636,11,7.727273,450.000000,6.454545,6.545455,6.111111,6.875000,1781.818182,3.636364,53.363636,11
1,2,2,35.8,4.0,2,1,234,3.0,915,6,0,0.0,510.0,20,1,59,1.9,1800.0,3.0,54.0,0,2,955.0,1,8,6,2,3,0,27334,110.0,3,2.0,1.125000,517.500000,5.500000,4.750000,5.333333,5.666667,1562.500000,1.000000,55.375000,8,1.125000,517.500000,5.500000,4.750000,5.333333,5.666667,1562.500000,1.000000,55.375000,8
2,3,2,36.2,2.0,3,1,78,2.0,3214,6,0,-6.0,482.0,20,1,59,1.8,1800.0,2.0,54.0,0,2,955.0,1,161,6,3,3,0,27335,110.1,2,1.0,3.583333,495.833333,2.583333,2.333333,2.500000,2.500000,1850.000000,2.000000,55.083333,12,3.583333,495.833333,2.583333,2.333333,2.500000,2.500000,1850.000000,2.000000,55.083333,12
3,1,2,36.7,1.0,1,1,95,1.0,6096,6,0,0.0,442.0,10,1,59,22.2,1800.0,1.0,52.0,0,2,955.0,1,23,6,4,3,0,27336,110.5,1,4.0,6.333333,442.666667,1.833333,2.000000,1.333333,1.333333,1666.666667,0.000000,52.000000,6,6.333333,442.666667,1.833333,2.000000,1.333333,1.333333,1666.666667,0.000000,52.000000,6
4,4,2,36.6,5.0,4,1,47,5.0,2932,6,0,-8.0,426.0,20,1,59,55.7,1800.0,6.0,54.0,0,2,955.0,1,217,6,5,3,0,27337,111.0,6,5.0,9.333333,437.666667,8.000000,8.333333,6.000000,6.333333,1683.333333,1.666667,53.333333,6,9.333333,437.666667,8.000000,8.333333,6.000000,6.333333,1683.333333,1.666667,53.333333,6


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92910 entries, 0 to 92909
Data columns (total 53 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Uni_num                   92910 non-null  int64  
 1   field_condition           92910 non-null  int64  
 2   ato_3_F                   92910 non-null  float64
 3   Corner_rank_3             45162 non-null  float64
 4   Hor_Num                   92910 non-null  int64  
 5   competition_count         92910 non-null  int64  
 6   Trainer                   92910 non-null  int64  
 7   Corner_rank_4             39573 non-null  float64
 8   Hor_name                  92910 non-null  int64  
 9   venue_area                92910 non-null  int64  
 10  Race_id                   92910 non-null  int64  
 11  Horse_weight_Flux         92893 non-null  float64
 12  Horse_weight              92910 non-null  float64
 13  Hor_sex_and_age           92910 non-null  int64  
 14  day_co

##Training with cross-validation

In [28]:
# データの並び順を元に分割する
folds = TimeSeriesSplit(n_splits=15)

X = df.drop(['Ranking','Race_Time','Corner_rank_1',
    'Corner_rank_2', 'Corner_rank_3', 'Corner_rank_4'],axis=1) #Leakageになりそうなものも含めて除外する
y = df['Ranking']

#評価結果格納用
oof_predictions = np.zeros(df.shape[0])

for i, (train_index, val_index) in enumerate(folds.split(df,groups=df['Race_id'])):
    print(f"Fold:{i+1} has started." )
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    #group引数用
    train_data = pd.concat([X_train,y_train],axis=1)
    train_groups = train_data.groupby('Race_id').size().to_frame('size')['size'].to_numpy()
    val_data = pd.concat([X_val,y_val],axis=1)
    val_groups = val_data.groupby('Race_id').size().to_frame('size')['size'].to_numpy()
    # print(train_groups)
    # print(val_groups)

    train_dataset = lgb.Dataset(X_train, y_train, group=train_groups)
    val_dataset = lgb.Dataset(X_val, y_val, reference=train_dataset, group=val_groups)

    params = {'task': 'train',
              'boosting_type': 'gbdt',
              'objective': 'lambdarank',
              'metric': 'ndcg',
              'nucg_eval_at': [5],
              'feature_pre_filter': False,
              'lambda_l1': 0.0, 'lambda_l2': 0.0,
              'num_leaves': 229,
              'feature_fraction': 0.7,
              'bagging_fraction': 1.0,
              'bagging_freq': 0,
              'min_child_samples': 20}

    model = lgb.train(params = params,
                    train_set=train_dataset,
                    valid_sets= [train_dataset,val_dataset],
                    num_boost_round =5000,
                    early_stopping_rounds=50,
                    verbose_eval = 50,
                    )
    oof_predictions[val_index] = model.predict(X_val,num_iteration=model.best_iteration)
    
    with open(f'pred_model_{i}.pkl', mode='wb') as f: #save each model
        pickle.dump(model,f)

result = pd.DataFrame(oof_predictions,columns=['predicted_class'])

df_result = pd.concat([df,result],axis=1)

Fold:1 has started.
Training until validation scores don't improve for 50 rounds.
[50]	training's ndcg@1: 1	training's ndcg@2: 0.997627	training's ndcg@3: 0.992237	training's ndcg@4: 0.989966	training's ndcg@5: 0.989195	valid_1's ndcg@1: 0.642586	valid_1's ndcg@2: 0.671709	valid_1's ndcg@3: 0.702534	valid_1's ndcg@4: 0.728745	valid_1's ndcg@5: 0.754263
Early stopping, best iteration is:
[48]	training's ndcg@1: 1	training's ndcg@2: 0.996309	training's ndcg@3: 0.991606	training's ndcg@4: 0.98941	training's ndcg@5: 0.98864	valid_1's ndcg@1: 0.643473	valid_1's ndcg@2: 0.666985	valid_1's ndcg@3: 0.698604	valid_1's ndcg@4: 0.730215	valid_1's ndcg@5: 0.755183
Fold:2 has started.
Training until validation scores don't improve for 50 rounds.
[50]	training's ndcg@1: 1	training's ndcg@2: 0.992109	training's ndcg@3: 0.986012	training's ndcg@4: 0.981988	training's ndcg@5: 0.981902	valid_1's ndcg@1: 0.682334	valid_1's ndcg@2: 0.701235	valid_1's ndcg@3: 0.73393	valid_1's ndcg@4: 0.765617	valid_1's nd

In [29]:
pd.options.display.max_rows=10000
df_result[['Ranking','predicted_class']].head(10000)

,Ranking,predicted_class
0,1,0.000000
1,2,0.000000
2,3,0.000000
3,4,0.000000
4,5,0.000000
5,6,0.000000
6,6,0.000000
7,5,0.000000
8,2,0.000000
9,7,0.000000
